In [1]:
import pandas as pd
import mysql.connector

In [2]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="rootroot"
)

In [3]:
cur = mydb.cursor()

In [4]:
cur.execute("SELECT * FROM sakila.actor")

In [5]:
actor = pd.DataFrame(cur.fetchall())
actor.columns = cur.column_names

<h3><b>Question 1</b></h3>

The store decides to run a promotion on family friendly movies for the Thanksgiving holiday week. Help the store manager identify which Category of Family Friendly films have the highest total Rental Orders along with the overall performance of Family Friendly catgories.

In [6]:
cur.execute(""" SELECT 
  name as category_name, 
  SUM(num_rentals) AS total_rental_orders 
FROM 
  (
    SELECT 
      f.title, 
      cat.name, 
      COUNT(re.rental_id) AS num_rentals 
    FROM 
      sakila.film f 
      INNER JOIN sakila.film_category fc ON f.film_id = fc.film_id 
      INNER JOIN sakila.category cat ON fc.category_id = cat.category_id 
      AND cat.name IN (
        'Animation', 'Children', 'Classics', 
        'Comedy', 'Family', 'Music'
      ) 
      INNER JOIN sakila.inventory inv ON inv.film_id = f.film_id 
      INNER JOIN sakila.rental re ON re.inventory_id = inv.inventory_id 
    GROUP BY 
      f.title, 
      cat.name
  ) AS distinct_result 
GROUP BY 
  name 
ORDER BY 
  SUM(num_rentals) DESC;""")

family_friendly_cat = pd.DataFrame(cur.fetchall())
family_friendly_cat.columns = cur.column_names

family_friendly_cat

,category_name,total_rental_orders
0,Animation,1166
1,Family,1096
2,Children,945
3,Comedy,941
4,Classics,939
5,Music,830


<b>Answer 1</b>

Of the 6 Family friendly film Categories, in terms of Total Number of Rental Orders we can see that the Animation category hasis the highest with 1166 Rental Orders.

<h3><b>Question 2</b></h3>

How many copies of the film Hunchback Impossible exist in the inventory system? 

In [7]:
cur.execute(""" SELECT 
  flm.title, 
  COUNT(*) number_in_inventory 
FROM 
  sakila.film flm 
  INNER JOIN sakila.inventory inv on flm.film_id = inv.film_id 
WHERE 
  LOWER(flm.title) = LOWER('Hunchback Impossible') 
GROUP BY 
  flm.title;""")

num_copies = pd.DataFrame(cur.fetchall())
num_copies.columns = cur.column_names

num_copies

,title,number_in_inventory
0,HUNCHBACK IMPOSSIBLE,6


<b>Answer 2</b>

There are 6 copies of the Hunchback Impossible available in the inventory system.

<h3><b>Question 3</b></h3>

The marketing manager wants to run an email marketing campaign for National Hispanic Heritage Month, for which he will need the names and email addresses of all Spanish customers. Help him retrieve this information.

In [8]:
cur.execute(""" SELECT 
  cus.first_name, 
  cus.last_name, 
  cus.email 
FROM 
  sakila.customer cus 
  INNER JOIN sakila.address adr ON cus.address_id = adr.address_id 
  INNER JOIN sakila.city cit ON adr.city_id = cit.city_id 
  INNER JOIN (
    SELECT 
      country_id 
    FROM 
      sakila.country 
    WHERE 
      country = 'Spain'
  ) cou on cit.country_id = cou.country_id 
ORDER BY 
  cus.first_name, 
  cus.last_name;""")

market_campaign = pd.DataFrame(cur.fetchall())
market_campaign.columns = cur.column_names

market_campaign

,first_name,last_name,email
0,CHRIS,BROTHERS,CHRIS.BROTHERS@sakilacustomer.org
1,JULIE,SANCHEZ,JULIE.SANCHEZ@sakilacustomer.org
2,MAE,FLETCHER,MAE.FLETCHER@sakilacustomer.org
3,NELSON,CHRISTENSON,NELSON.CHRISTENSON@sakilacustomer.org
4,WESLEY,BULL,WESLEY.BULL@sakilacustomer.org


<h3><b>Question 4</b></h3>

What are the top 10 film categories based on the average rental duration of films in each category ?

In [9]:
cur.execute(""" SELECT 
  category.category_id, 
  category.name AS category_name, 
  AVG(film.rental_duration) AS avg_rental_duration 
FROM 
  sakila.film 
  INNER JOIN sakila.film_category 
  INNER JOIN sakila.category ON film.film_id = film_category.film_id 
  AND film_category.category_id = category.category_id 
GROUP BY 
  category.category_id 
ORDER BY 
  avg_rental_duration DESC
LIMIT 
  10;""")

avg_rental_duration = pd.DataFrame(cur.fetchall())
avg_rental_duration.columns = cur.column_names

avg_rental_duration

,category_id,category_name,avg_rental_duration
0,16,Travel,5.3509
1,12,Music,5.2353
2,8,Family,5.1739
3,9,Foreign,5.1096
4,7,Drama,5.0806
5,4,Classics,5.0702
6,10,Games,5.0656
7,3,Children,5.0333
8,1,Action,4.9531
9,5,Comedy,4.9310


<b>Answer 4</b>

The average rental duration across all categories is approximately 5 minutes

<h3><b>Question 5</b></h3>

We want to find out how the two stores compare in their count of rental orders during every month for all the years we have data for. Write a query that returns the store ID for the store, the year and month and the number of rental orders each store has fulfilled for that month.


In [10]:
cur.execute(""" SELECT 
  MONTH(re.rental_date) rental_month, 
  YEAR(re.rental_date) rental_year, 
  sto.store_id, 
  COUNT(re.rental_id) AS count_rentals 
FROM 
  sakila.rental re 
  INNER JOIN sakila.staff sta ON sta.staff_id = re.staff_id 
  INNER JOIN sakila.store sto ON sto.store_id = sta.store_id 
GROUP BY 
  sto.store_id, 
  rental_month, 
  rental_year 
ORDER BY 
  count_rentals DESC;""")

store_level = pd.DataFrame(cur.fetchall())
store_level.columns = cur.column_names

store_level

,rental_month,rental_year,store_id,count_rentals
0,7,2005,2,3367
1,7,2005,1,3342
2,8,2005,1,2892
3,8,2005,2,2794
4,6,2005,1,1163
5,6,2005,2,1148
6,5,2005,2,598
7,5,2005,1,558
8,2,2006,2,97
9,2,2006,1,85


<b>Answer 5</b>

The 2 stores have comparable sales and it can be seen that July and August of 2005 are the months with the most number of rentals 